In [ ]:
!pip install openai
!pip install aiofiles aiohttp



In [ ]:
# OPENAI API image similarity comparison between two images

import openai
import base64

# Initialize OpenAI API client
client = openai.OpenAI(api_key="YOUR_API_KEY_HERE")  # Replace with your key

# Function to encode image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Encode both images
image_path_1 = "IMAGE1_FOLDER"
image_path_2 = "IMAGE2_FOLDER"

base64_image_1 = encode_image(image_path_1)
base64_image_2 = encode_image(image_path_2)

# Ask GPT-4o to compare and score similarity
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": """
You will be shown two images. Compare their **visual similarity** based on content, layout, and overall appearance. Give a similarity score **from 0 to 100**, where:
- 100 means almost identical,
- 0 means completely different.

Only return the score and a brief justification. Format:
Similarity: XX%
Reason: ...
"""},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image_1}"}},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image_2}"}}
            ]
        }
    ],
    max_tokens=300
)

# Output result
print("\n🟩 Similarity Result:")
print(response.choices[0].message.content.strip())



🟩 Similarity Result:
Similarity: 85%
Reason: Both images depict colorful parrots surrounded by jungle foliage. They have similar vibrant colors and lighting, and the overall composition is alike, with the parrot positioned on a branch. Differences lie in the pose and slight variations in background elements.


In [ ]:
# OPENAI API image similarity comparison based on weights

import openai
import base64
import json
import re

# Define weights
weights = {
    "main_objects": 0.35,
    "main_object_attributes": 0.15,
    "location": 0.15,
    "action": 0.10,
    "attributes": 0.15,
    "background": 0.10
}

# Function to encode image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Encode both images
image_path_1 = "IMAGE1_PATH"
image_path_2 = "IMAGE2_PATH"

base64_image_1 = encode_image(image_path_1)
base64_image_2 = encode_image(image_path_2)

# Build the GPT prompt
comparison_prompt = """
Compare the two images on the following categories and give each a score from 0 to 100:

- main_objects
- main_object_attributes
- location
- action
- attributes
- background

Respond with ONLY a valid JSON object. No explanation, no markdown.

Example format:
{
  "main_objects": 85,
  "main_object_attributes": 70,
  "location": 90,
  "action": 60,
  "attributes": 75,
  "background": 50
}
"""

# Call OpenAI API
client = openai.OpenAI(api_key="YOUR_API_KEY_HERE")  # Replace with your actual API key

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": comparison_prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image_1}"}},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image_2}"}}
            ]
        }
    ],
    max_tokens=300
)

# Process GPT response
result_text = response.choices[0].message.content.strip()
print("🔍 GPT Raw Response:")
print(result_text)

# Try to extract and parse JSON
try:
    match = re.search(r"\{.*\}", result_text, re.DOTALL)
    if not match:
        raise ValueError("No JSON object found in GPT response.")

    json_text = match.group()
    category_scores = json.loads(json_text)

    # Compute final score
    final_score = sum(category_scores[category] * weights[category] for category in weights)

    print("\n🟩 Final Weighted Similarity Score:")
    print(f"Similarity: {round(final_score, 2)}%")

    print("\n🔍 Detailed Breakdown:")
    for category in weights:
        score = category_scores[category]
        weight = weights[category]
        weighted = score * weight
        print(f"{category}: {score} * {weight} = {weighted:.2f}")

except json.JSONDecodeError as e:
    print("\n❌ JSON Decode Error:")
    print(e)
except Exception as e:
    print("\n❌ Unexpected Error:")
    print(e)


🔍 GPT Raw Response:
{
  "main_objects": 95,
  "main_object_attributes": 90,
  "location": 85,
  "action": 50,
  "attributes": 80,
  "background": 75
}

🟩 Final Weighted Similarity Score:
Similarity: 84.0%

🔍 Detailed Breakdown:
main_objects: 95 * 0.35 = 33.25
main_object_attributes: 90 * 0.15 = 13.50
location: 85 * 0.15 = 12.75
action: 50 * 0.1 = 5.00
attributes: 80 * 0.15 = 12.00
background: 75 * 0.1 = 7.50


In [ ]:
# OPENAI API image similarity comparison based on weights for a folder

import openai
import base64
import json
import re
import os
import csv

# ====== Config ======
image_folder = "YOUR_IMAGE_FOLDER"  # 🔁 CHANGE THIS
output_csv = "similarity_scores.csv"
openai.api_key = "YOUR_API_KEY"  # 🔁 YOUR API KEY HERE

# ====== Weights ======
weights = {
    "main_objects": 0.35,
    "main_object_attributes": 0.15,
    "location": 0.15,
    "action": 0.10,
    "attributes": 0.15,
    "background": 0.10
}

# ====== Prompt Template ======
comparison_prompt = """
Compare the two images on the following categories and give each a score from 0 to 100:

- main_objects
- main_object_attributes
- location
- action
- attributes
- background

Respond with ONLY a valid JSON object. No explanation, no markdown.

Example format:
{
  "main_objects": 85,
  "main_object_attributes": 70,
  "location": 90,
  "action": 60,
  "attributes": 75,
  "background": 50
}
"""

# ====== Helper Functions ======
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def find_row_image_pairs(folder):
    pairs = []
    for file in os.listdir(folder):
        if file.endswith("_student.png") and file.startswith("row_"):
            row_id = file.replace("_student.png", "")
            student_path = os.path.join(folder, f"{row_id}_student.png")
            teacher_path = os.path.join(folder, f"{row_id}_teacher.png")
            if os.path.exists(teacher_path):
                pairs.append((row_id, student_path, teacher_path))
    return sorted(pairs, key=lambda x: int(x[0].split("_")[1]))  # Sort by row number

def calculate_similarity_score(base64_img1, base64_img2):
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": comparison_prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_img1}"}},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_img2}"}}
                ]
            }
        ],
        max_tokens=300
    )

    result = response.choices[0].message.content.strip()
    match = re.search(r"\{.*\}", result, re.DOTALL)
    if not match:
        raise ValueError("No JSON object found in GPT response.")
    scores = json.loads(match.group())
    return round(sum(scores[cat] * weights[cat] for cat in weights), 2)

# ====== Run ======
image_pairs = find_row_image_pairs(image_folder)
results = []

for row_id, student_path, teacher_path in image_pairs:
    try:
        print(f"🔍 Comparing {row_id}...")
        base64_student = encode_image(student_path)
        base64_teacher = encode_image(teacher_path)
        score = calculate_similarity_score(base64_teacher, base64_student)
        print(f"✅ {row_id}: {score}%")
        results.append([row_id, score])
    except Exception as e:
        print(f"❌ Error processing {row_id}: {e}")
        results.append([row_id, "Error"])

# ====== Save to CSV ======
with open(output_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Row", "Similarity Score (%)"])
    writer.writerows(results)

print(f"\n✅ All results saved in: {output_csv}")


🔍 Comparing row_1...
✅ row_1: 84.5%
🔍 Comparing row_2...
✅ row_2: 84.75%
🔍 Comparing row_3...
✅ row_3: 83.0%
🔍 Comparing row_4...
✅ row_4: 87.0%
🔍 Comparing row_5...
✅ row_5: 86.0%
🔍 Comparing row_6...
✅ row_6: 87.15%
🔍 Comparing row_7...
✅ row_7: 84.25%
🔍 Comparing row_8...
✅ row_8: 82.5%
🔍 Comparing row_9...
✅ row_9: 87.15%
🔍 Comparing row_10...
✅ row_10: 84.5%
🔍 Comparing row_11...
✅ row_11: 88.9%
🔍 Comparing row_12...
✅ row_12: 83.6%
🔍 Comparing row_13...
✅ row_13: 80.75%
🔍 Comparing row_14...
✅ row_14: 84.0%
🔍 Comparing row_15...
✅ row_15: 89.25%
🔍 Comparing row_16...
✅ row_16: 88.0%
🔍 Comparing row_17...
✅ row_17: 85.15%
🔍 Comparing row_18...
✅ row_18: 91.0%
🔍 Comparing row_19...
✅ row_19: 84.25%
🔍 Comparing row_20...
✅ row_20: 87.15%

✅ All results saved in: similarity_scores.csv
